In [1]:
import pandas as pd

from fuzzywuzzy import process

Loading Original Data

In [2]:
# From Kaggle
df_naturalDisasters = pd.read_csv("..\\Data_Sets\\originalData\\natural-disasters.csv")

# From Scimagojr
df_academicRelevance = pd.read_csv("..\\Data_Sets\\originalData\\academic_relevance_1996-2022.csv")

# From OurWorldInData
df_HDI = pd.read_csv("..\\Data_Sets\\originalData\\human-development-index.csv")
df_migrants = pd.read_csv("..\\Data_Sets\\originalData\\migrant-stock-total.csv")
df_naturalResources = pd.read_csv("..\\Data_Sets\\originalData\\naturalResourcesWealth.csv")
df_educationQuality = pd.read_csv("..\\Data_Sets\\originalData\\education_quality_by_outcome.csv")

# From World Bank
df_arableLand = pd.read_csv("..\\Data_Sets\\originalData\\arableLand.csv", skiprows=4)
df_qualifiedLaborForce = pd.read_csv("..\\Data_Sets\\originalData\\Labor_force_with_advanced_education.csv", skiprows=4)

Loading Data processed in other notebooks

In [3]:
df_climateData = pd.read_csv("..\\Data_Sets\\processed\\addData_climateData.csv")
df_countriesSize_regions = pd.read_csv("..\\Data_Sets\\processed\\addData_countriesSize_Regions.csv")
df_fromWiki = pd.read_csv("..\\Data_Sets\\processed\\addData_fromWiki.csv")
df_economicFreedom = pd.read_csv("..\\Data_Sets\\processed\\economicData_1995-2022.csv")